# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [ ]:
!pip install citipy

In [ ]:
#Specify necessary dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

#Get unique weather api key from the api_keys python file
from api_keys import weather_api_key

# Import citipy after pip install to find out the cities longitude and latitude coordinatesx
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [ ]:
# Empty list for lat and lon data
lat_lngs = []

# Empty list for all the cities
cities = []

# Range of values for longitude and latitude
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create random longitude and latitude
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

#For every latitude and longitude in the list
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    #for each city, add to the cities list
    if city not in cities:
        cities.append(city)

#print the total number of cities
print(f"Number of cities in the list: {len(cities)}")

In [ ]:
cities

---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [ ]:
url = f"http://api.openweathermap.org/data/2.5/weather?units=metric&appid={weather_api_key}"

In [ ]:
#Define base url with unique weather api key
url = f"http://api.openweathermap.org/data/2.5/weather?units=metric&appid={weather_api_key}"

#Create an empty list to get the weather data for each city
city_data = []

#Print to tell the user that data is being gathered
print("Beginning Data Retrieval     ")
print("-----------------------------")

#Create counters for record and set
record_count = 1
set_count = 1

#Loop through all the cities in city_data to get weather data
for i, city in enumerate(cities):
        
    #Group cities in groups of 50 each
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    #Create endpoint URL for each city
    city_url = f"{url}&q={city}"
    
    #Keep track the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    #Increment the record count by 1
    record_count += 1

    #Run an API request/call for each of the cities
    try:
        #Get the request and convert to json to get the raw data
        city_weather = requests.get(city_url).json()

        #Take the latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        #Append the previous values into the city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    #If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
#Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
#Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

#Show the count
city_data_df.count()

In [ ]:
#Show the dataframe for the first five rows
city_data_df.head()

In [ ]:
#Export the city data dataframe to a csv file named cities.csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
#Create a scatter plot for latitude vs. temperature
plt.scatter(
    city_data_df["Lat"],
    city_data_df["Max Temp"],
    edgecolor="red",
    linewidths=1,
    marker="o",
    alpha=0.8,
    label="Cities")

#Make a title, x label title, and y label title
plt.title(f"City Max Latitude vs. Temperature")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (C)")

#Save the plot to Fig1.png
plt.savefig("output_data/Fig1.png")

# Show the plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
#Create a scatter plot for latitude vs. humidity
plt.scatter(
    city_data_df["Lat"],
    city_data_df["Humidity"],
    edgecolor="green",
    linewidths=1,
    marker="o",
    alpha=0.8,
    label="Cities")

#Make a title, x label title, and y label title
plt.title(f"City Max Latitude vs. Humidity")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

#Save the plot to Fig2.png
plt.savefig("output_data/Fig2.png")

# Show the plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
#Create a scatter plot for latitude vs. cloudiness
plt.scatter(
    city_data_df["Lat"],
    city_data_df["Cloudiness"],
    edgecolor="orange",
    linewidths=1,
    marker="o",
    alpha=0.8,
    label="Cities")

#Make a title, x label title, and y label title
plt.title(f"City Max Latitude vs. Cloudiness")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

# Save the plot to Fig3.png
plt.savefig("output_data/Fig3.png")

# Show the plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
#Create a scatter plot for latitude vs. wind speed
plt.scatter(
    city_data_df["Lat"],
    city_data_df["Wind Speed"],
    edgecolor="purple",
    linewidths=1,
    marker="o",
    alpha=0.8,
    label="Cities")

#Make a title, x label title, and y label title
plt.title(f"City Max Latitude vs. Wind Speed")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (%)")

# Save the plot to Fig4.png
plt.savefig("output_data/Fig4.png")

# Show the plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
#Define a function to make the Linear Regression plots
def plot_linear_regression(x_value, y_value, title, text_coord):
    
    (slope, intercept, rvalue, pvalue, stderr)= linregress(x_value, y_value)
    regress_value = x_value * slope + intercept
    line_eq = f"y = {round(slope, 2)}x + {round(intercept, 2)}"
    
    plt.scatter(x_value, y_value)
    plt.plot(x_value, regress_value, "r-")
    plt.annotate(line_eq, text_coord, fontsize=15, color="red")
    plt.xlabel("Lat")
    plt.ylabel(title)
    print(f"The r-value is {rvalue ** 2}")
    plt.show()

In [ ]:
#Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df[city_data_df["Lat"] >=0]

#Display the data
northern_hemi_df.head()

In [ ]:
#Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df[city_data_df["Lat"]<0]

#Display the data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
#Linear regression on Northern Hemisphere
x_values= northern_hemi_df["Lat"]
y_values= northern_hemi_df["Max Temp"]
plot_linear_regression(x_values, y_values, "Max Temp", (-40, 30))

In [ ]:
# Linear regression on Southern Hemisphere
x_values= southern_hemi_df["Lat"]
y_values= southern_hemi_df["Max Temp"]
plot_linear_regression(x_values, y_values, "Max Temp", (-55, 5))

**Discussion about the linear relationship:** YOUR RESPONSE HERE
The regression linear graphs show that as latitude increases for the Northern Hemisphere, the max temperature goes down. For the Southern Hemisphere, when latitude increases, the max temperature goes up.

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
#Linear regression on Northern Hemisphere
x_values= northern_hemi_df["Lat"]
y_values= northern_hemi_df["Humidity"]
plot_linear_regression(x_values, y_values, "Humidity", (50, 20))

In [ ]:
#Linear regression on Southern Hemisphere
x_values= southern_hemi_df["Lat"]
y_values= southern_hemi_df["Humidity"]
plot_linear_regression(x_values, y_values, "Humidity", (-50, 30))

**Discussion about the linear relationship:** YOUR RESPONSE HERE
The regression linear graphs show that the humidity remains relatively constant for any value of latitude for both the Northern and Southern Hemispheres.

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
#Linear regression on Northern Hemisphere
x_values= northern_hemi_df["Lat"]
y_values= northern_hemi_df["Cloudiness"]
plot_linear_regression(x_values, y_values, "Cloudiness", (40, 20))

In [ ]:
#Linear regression on Southern Hemisphere
x_values= southern_hemi_df["Lat"]
y_values= southern_hemi_df["Cloudiness"]
plot_linear_regression(x_values, y_values, "Cloudiness", (-50, 20))

**Discussion about the linear relationship:** YOUR RESPONSE HERE
The regression linear graphs show that cloudiness basically stays constant for the Southern Hemisphere. On the other hand, the Northern Hemisphere shows some decrease in cloudiness as latitude increases.

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
#Linear regression on Northern Hemisphere
x_values= northern_hemi_df["Lat"]
y_values= northern_hemi_df["Wind Speed"]
plot_linear_regression(x_values, y_values, "Wind Speed", (10, 15))

In [ ]:
#Linear regression on Southern Hemisphere
x_values= southern_hemi_df["Lat"]
y_values= southern_hemi_df["Wind Speed"]
plot_linear_regression(x_values, y_values,  "Wind Speed", (-50, 10))

**Discussion about the linear relationship:** YOUR RESPONSE HERE
The regression linear graphs show that wind speed drops when latitude goes up for the Southern Hemisphere. The wind speed remains constant for the most part in the Northern Hemisphere.